In [1]:
%pip install --upgrade baolib



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import tempfile
from pathlib import Path

from baolib.bao import (
    Access,
    AccessChange,
    Vault,
    DB,
    Store,
    Replica,
    Message,
    Mailbox,
    local_store,
    set_bao_log_level,
    newKeyPair,
)

set_bao_log_level('debug')

DEBUG [20:02:00] (+0ms)        END[main.bao_setLogLevel] export.go:108 - level debug


INFO  [20:02:00] (+26ms) core.syncTime[time.go:34] - clock offset 12.309564ms from time.google.com 


# Introduction
Bao is a secure storage library for sharing encrypted data. This notebook shows the Python binding (`baolib`) API.


## Identity
Create identities with `newKeyPair()` which returns a tuple of (publicID, privateID).

In [3]:
alice, alice_secret = newKeyPair()

print('Owner public ID:', alice)

keys = alice_secret.decode()
print(f'secp256k1 key: {keys["cryptKey"]}, ed25519 key: {keys["signKey"]}') 


Owner public ID: AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk=
secp256k1 key: CpqYq8VobwieJEsF0NXAhHqYm9Y8nENmREsw_rAITZw=, ed25519 key: xdxkEWO9mLABRODxI0cE5EnGAlEu5Mcb0kNv8G76C7E=


DEBUG [20:02:01] (+410ms)        START[main.bao_security_newKeyPair] export.go:177 - 
DEBUG [20:02:01] (+0ms)          START[security.NewPrivateID] identity.go:45 - generating new private ID
DEBUG [20:02:01] (+1ms)          END[security.NewPrivateID] identity.go:57 - 
DEBUG [20:02:01] (+1ms)        END[main.bao_security_newKeyPair] export.go:185 - generated new key pair
DEBUG [20:02:01] (+1ms)        START[main.bao_security_decodePrivateID] export.go:291 - id len 88
DEBUG [20:02:01] (+0ms)        END[main.bao_security_decodePrivateID] export.go:303 - decoded ID into key components


## Local database
Use a local SQLite database to track metadata. Here we start with a clean DB in a temp directory.


In [4]:
temp_root = Path(tempfile.gettempdir()) / 'bao_python_sample'
temp_root.mkdir(parents=True, exist_ok=True)

def new_db_path(name: str) -> DB:
    db_path = temp_root / name
    if db_path.exists():
        db_path.unlink()
    return str(db_path)

db_path = new_db_path('bao.db')
db = DB('sqlite3', db_path)
print('Database at', db_path)


Database at /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db


DEBUG [20:02:02] (+1036ms)        START[main.bao_db_open] export.go:311 - driver sqlite3
DEBUG [20:02:02] (+0ms)         START[sqlx.Open] db.go:35 - Opening SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3
DEBUG [20:02:02] (+0ms)          START[sqlx.OpenEngine] engine_native.go:31 - Opening database /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3
DEBUG [20:02:02] (+0ms) Database file does not exist, creating new file at: /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db
DEBUG [20:02:02] (+0ms)          END[sqlx.OpenEngine] engine_native.go:56 - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3
INFO  [20:02:02] (+0ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao.db with driver sqlite3
DEBUG [20:02:02] (+0ms)       

## Create a vault
A vault combines the database, an identity, and a storage URL. The storage can be local (`file://`), S3, SFTP, etc.


In [5]:
store = Store(local_store('vault', str(temp_root / 'vault')))
vault = Vault.create(Vault.users, alice_secret, store, db)
vault

DEBUG [20:02:03] (+1136ms)        START[main.bao_store_open] export.go:453 - store config {"id": "vault", "type": "local", "local": {"base": "file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault"}}
DEBUG [20:02:03] (+0ms)          START[store.OpenLocal] local.go:26 - Opening local store.with URL: file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault
DEBUG [20:02:03] (+0ms)          END[store.OpenLocal] local.go:31 - Opened local store.with path: file:///var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault
DEBUG [20:02:03] (+0ms)        END[main.bao_store_open] export.go:469 - opened store vault
DEBUG [20:02:03] (+0ms)        START[main.bao_vault_create] export.go:579 - db handle 1
DEBUG [20:02:03] (+0ms)         START[vault.Create] create.go:22 - creating vault for url vault
DEBUG [20:02:03] (+10ms)           START[store.(*Local).ReadDir] local.go:116 - Reading directory: users
DEBUG [20:02:03] (+10ms)           EN

vault|users

## Access control
Grant permissions to groups. Built-in groups include `admins`, `users`, and `public`. Permissions are bitmasks (`Access.read`, `Access.write`, `Access.admin`).


In [6]:
bob, bob_secret = newKeyPair()
vault.sync_access([
    AccessChange(bob, Access.read),
])

accesses = vault.get_accesses()
for user, access in accesses.items():
    print(f"User {user} has access level: {access}")

print('Bob specific access:', vault.get_access(bob))

User Ar7lcVNdDh9x4OVl0XYlPtPQliMZaH3X3l5ktnomOMVx7m6NnScQu1THO0j_quIWJCV1RM9NkdH60KiTiV6GTr8= has access level: 1
User AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk= has access level: 7
Bob specific access: 1


DEBUG [20:02:04] (+1455ms)        START[main.bao_security_newKeyPair] export.go:177 - 
DEBUG [20:02:04] (+0ms)          START[security.NewPrivateID] identity.go:45 - generating new private ID
DEBUG [20:02:04] (+0ms)          END[security.NewPrivateID] identity.go:57 - 
DEBUG [20:02:04] (+0ms)        END[main.bao_security_newKeyPair] export.go:185 - generated new key pair
DEBUG [20:02:04] (+0ms)        START[main.bao_vault_syncAccess] export.go:666 - handle 1
DEBUG [20:02:04] (+0ms)         START[vault.(*Vault).SyncAccess] access.go:16 - syncing access changes [{Ar7lcVNdDh9x4OVl0XYlPtPQliMZaH3X3l5ktnomOMVx7m6NnScQu1THO0j_quIWJCV1RM9NkdH60KiTiV6GTr8= R}] with options 0
DEBUG [20:02:04] (+0ms)          START[vault.(*Vault).convertToChanges] access.go:57 - staging 1 access changes
DEBUG [20:02:04] (+0ms)           START[vault.(*Vault).GetAccesses] access.go:172 - group users
DEBUG [20:02:04] (+0ms)           END[vault.(*Vault).GetAccesses] access.go:194 - 1 users
DEBUG [20:02:04] (+0ms)   

## Open an existing vault
Anyone with the vault URL, their private ID, and the creator's public ID can reopen the vault.


In [13]:
vault = Vault.open(Vault.users, alice_secret, alice, store, db)
print('Vault opened with access level:', vault.get_access(alice))
vault

Vault opened with access level: 7


DEBUG [20:02:45] (+25051ms)        START[main.bao_vault_open] export.go:616 - handle 1
DEBUG [20:02:45] (+0ms)         START[vault.Open] open.go:22 - opening vault with store URL vault
DEBUG [20:02:45] (+0ms)          START[vault.(*Vault).GetAccess] access.go:200 - user AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk=
DEBUG [20:02:45] (+0ms)          END[vault.(*Vault).GetAccess] access.go:212 - access 7
DEBUG [20:02:45] (+0ms)          START[vault.(*Vault).startHousekeeping] housekeeping.go:101 - starting housekeeping
DEBUG [20:02:45] (+0ms)          END[vault.(*Vault).startHousekeeping] housekeeping.go:113 - housekeeping started
INFO  [20:02:45] (+0ms) vault.Open[open.go:85] - successfully opened vault vault|users
DEBUG [20:02:45] (+0ms)         END[vault.Open] open.go:86 - 
DEBUG [20:02:45] (+0ms)          START[vault.(*Vault).syncBlockChain] blockchain.go:364 - 
DEBUG [20:02:45] (+0ms)           START[vault.(*Vault).importBlocksFromStorage] b

vault|users

DEBUG [20:12:03] (+558269ms)     START[vault.(*Vault).housekeeping] housekeeping.go:82 - starting housekeeping
DEBUG [20:12:03] (+558269ms)      START[vault.(*Vault).syncBlockChain] blockchain.go:364 - 
DEBUG [20:12:03] (+558270ms)       START[vault.(*Vault).importBlocksFromStorage] blockchain.go:251 - 
DEBUG [20:12:03] (+558270ms)        START[vault.(*Vault).getLastBlockHash] blockchain.go:187 - 
DEBUG [20:12:03] (+558271ms)        END[vault.(*Vault).getLastBlockHash] blockchain.go:197 - hash de3aea9e49cb16d2fcee7b5f2a6da7769a2c0c2408cd7dd90cb469339038fe9f9b8528c1746719fea816e201864eadb7dfaaa5589441ad557b70251303e3d089
DEBUG [20:12:03] (+558271ms)        START[vault.(*Vault).importBlockFromStorage] blockchain.go:202 - name 3jrqnknLFtL87ntfKm2ndposDCQIzX3ZDLRpM5A4_p-bhSjBdGcZ_qgW4gGGTq2336qlWJRBrVV7cCUTA-PQiQ
DEBUG [20:12:03] (+558271ms)          START[store.(*Local).Read] local.go:40 - name users/blockchain/3jrqnknLFtL87ntfKm2ndposDCQIzX3ZDLRpM5A4_p-bhSjBdGcZ_qgW4gGGTq2336qlWJRBrVV7cC

## Files and data
Write and read files using group permissions.


In [8]:
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

vault.write('docs/hello.txt', src=str(data_path)) # write file to vault
ls = vault.read_dir('docs', None, 0, 10)                        # list files in docs/

print('Files in docs/:', [l['name'] if isinstance(l, dict) else l.name for l in ls])

Files in docs/: ['hello.txt']


DEBUG [20:02:09] (+0ms)        START[main.bao_vault_write] export.go:947 - called with sH: 2, dest: docs/hello.txt, source: /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt, options: 0
DEBUG [20:02:09] (+0ms)         START[vault.(*Vault).Write] write.go:169 - dest docs/hello.txt, source /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt, options 0
DEBUG [20:02:09] (+0ms)          START[vault.(*Vault).writeRecord] write.go:34 - writing record to docs/hello.txt
DEBUG [20:02:09] (+0ms)           START[vault.(*Vault).getLastKeyFromDB] keys.go:15 - getting last key for group users
DEBUG [20:02:09] (+0ms)           END[vault.(*Vault).getLastKeyFromDB] keys.go:24 - successfully got last key for group users: id=409423307949461504, key=db7c89f521bbb722fc41a61575c2049e3ab2222116fa6d67095ea551eba5e357
DEBUG [20:02:09] (+0ms)           START[vault.getSegmentDir] name.go:43 - segmentInterval 0s
DEBUG [20:02:09] (+0ms)           END[vault.getSegm

DEBUG [20:02:10] (+999ms)      START[store.(*Local).Write] local.go:88 - Writing file: users/data/.last_change
DEBUG [20:02:10] (+999ms)       START[store.createDir] local.go:78 - Creating directory: file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data/.last_change
DEBUG [20:02:10] (+999ms)       END[store.createDir] local.go:83 - successfully created directory: file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data
DEBUG [20:02:10] (+999ms)      END[store.(*Local).Write] local.go:111 - wrote 0 bytes to file file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data/.last_change


In [9]:
out_path = temp_root / 'hello.out'
vault.read('docs/hello.txt', str(out_path))
print('written content:', out_path.read_text())


written content: Hello from Bao!


DEBUG [20:02:11] (+0ms)        START[main.bao_vault_read] export.go:925 - called with sH: 2, name: docs/hello.txt, dest: /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.out, options: 0
DEBUG [20:02:11] (+0ms)         START[vault.(*Vault).Read] read.go:79 - reading file docs/hello.txt to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.out
DEBUG [20:02:11] (+0ms)          START[vault.(*Vault).queryFileByName] file.go:69 - querying file by name docs/hello.txt
DEBUG [20:02:11] (+0ms)          END[vault.(*Vault).queryFileByName] file.go:108 - successfully retrieved info for file {1 docs/hello.txt users 15 167 2026-02-03 20:02:09.833 +0100 CET false 0 [] /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt 409423307949461504 users/data/20260203000000 7jg77xuo AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk=}
DEBUG [20:02:11] (+0ms)          START[vault.(*Vault).UpdateFileFlags] r

I/O operations can be asynchronous by using the option _async_operation_ or _scheduled_operation_

In [10]:
import time
data_path = temp_root / 'hello.txt'
data_path.write_text('Hello from Bao!')

file = vault.write('docs/hello.txt', src=str(data_path), options=Vault.async_operation) # write file to vault
start = time.perf_counter_ns()

# other operations can be done here while write is in progress
vault.wait_files([file.file_id])      # wait for write to complete
print(f'File written in {(time.perf_counter_ns() - start) / 1_000_000 } ms')

File written in 1.615209 ms


DEBUG [20:02:15] (+0ms)        START[main.bao_vault_write] export.go:947 - called with sH: 2, dest: docs/hello.txt, source: /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt, options: 1
DEBUG [20:02:15] (+0ms)         START[vault.(*Vault).Write] write.go:169 - dest docs/hello.txt, source /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt, options 1
DEBUG [20:02:15] (+0ms)          START[vault.(*Vault).writeRecord] write.go:34 - writing record to docs/hello.txt
DEBUG [20:02:15] (+0ms)           START[vault.(*Vault).getLastKeyFromDB] keys.go:15 - getting last key for group users
DEBUG [20:02:15] (+0ms)           END[vault.(*Vault).getLastKeyFromDB] keys.go:24 - successfully got last key for group users: id=409423307949461504, key=db7c89f521bbb722fc41a61575c2049e3ab2222116fa6d67095ea551eba5e357
DEBUG [20:02:15] (+0ms)           START[vault.getSegmentDir] name.go:43 - segmentInterval 0s
DEBUG [20:02:15] (+0ms)           END[vault.getSegm

DEBUG [20:02:16] (+1001ms)      START[store.(*Local).Write] local.go:88 - Writing file: users/data/.last_change
DEBUG [20:02:16] (+1001ms)       START[store.createDir] local.go:78 - Creating directory: file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data/.last_change
DEBUG [20:02:16] (+1001ms)       END[store.createDir] local.go:83 - successfully created directory: file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data
DEBUG [20:02:16] (+1001ms)      END[store.(*Local).Write] local.go:111 - wrote 0 bytes to file file:/var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/vault/users/data/.last_change


In [11]:
file = vault.stat('docs/hello.txt')
print(f'File stats: name = {file.name}', f'size = {file.size} bytes', f'flags = {file.flags}', 
          f'author = {file.author_id}')

#vault.delete('docs/hello.txt')

File stats: name = docs/hello.txt size = 15 bytes flags = 0 author = AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk=


DEBUG [20:02:16] (+0ms)        START[main.bao_vault_stat] export.go:883 - called with sH: 2, name: docs/hello.txt
INFO  [20:02:16] (+0ms) vault.(*Vault).Stat[stat.go:40] - successfully got file info for docs/hello.txt: id=2, modTime=2026-02-03 20:02:15.323 +0100 CET, size=15, allocated=167, isDir=false
DEBUG [20:02:16] (+0ms)        END[main.bao_vault_stat] export.go:898 - successful statistic for file docs/hello.txt in vault 2%!(EXTRA vault.File={2 docs/hello.txt users 15 167 2026-02-03 20:02:15.323 +0100 CET false 0 [] /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.txt 409423307949461504 users/data/20260203000000 7jgxzw2o AyWndf9euJbInoTss2iEpFjKQOPcY-QnINg3N-6fUo3m0AQ_vrFTnvp1FtZxzOMW0b_D_fk5PSu2gjp7PsBRkhk=})


## Exchange with another peer
Create a new database to simulate a different peer. Read the content.


In [12]:
#set_bao_log_level('info')

db2_path = new_db_path('bao2.db')
db2 = DB('sqlite3', db2_path)
vault2 = Vault.open(Vault.users, bob_secret, alice, store, db2)
print('Bob opens vault with access level:', vault2.get_access(bob))
ls = vault2.read_dir('docs', None, 0, 10)

print('Alice sees files in docs/:', [l.name for l in ls])

Bob opens vault with access level: 1
Alice sees files in docs/: ['hello.txt']


DEBUG [20:02:20] (+3371ms)        START[main.bao_db_open] export.go:311 - driver sqlite3
DEBUG [20:02:20] (+0ms)         START[sqlx.Open] db.go:35 - Opening SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db with driver sqlite3
DEBUG [20:02:20] (+0ms)          START[sqlx.OpenEngine] engine_native.go:31 - Opening database /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db with driver sqlite3
DEBUG [20:02:20] (+0ms) Database file does not exist, creating new file at: /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db
DEBUG [20:02:20] (+0ms)          END[sqlx.OpenEngine] engine_native.go:56 - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db with driver sqlite3
INFO  [20:02:20] (+0ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/bao2.db with driver sqlite3
DEBUG [20:02:20] (+0ms)  

In [13]:
vault2.read('docs/hello.txt', str(temp_root / 'hello.collab.out'))
(temp_root / 'hello.collab.out').read_text()

INFO  [04:33:20] (+3ms) vault.(*Vault).Read[read.go:120] - successfully read file docs/hello.txt to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.collab.out in 3.265916ms


'Hello from Bao!'

In [15]:
vault2.write('docs/hello.txt', src=str(data_path))
versions =vault2.versions('docs/hello.txt')

vault2.read(versions[-1].name, str(temp_root / 'hello.versioned.out'))
(temp_root / 'hello.versioned.out').read_text()

INFO  [04:34:06] (+0ms) vault.(*Vault).WaitFiles[wait_files.go:36] - Successfully synchronized vault with store ID vault|users in 30.708µs
INFO  [04:34:06] (+0ms) vault.(*Vault).Versions[versions.go:58] - successfully got file versions for hello.txt: 4
INFO  [04:34:06] (+1ms) vault.(*Vault).Read[read.go:120] - successfully read file docs/hello.txt:3 to /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/hello.versioned.out in 1.827041ms


'Hello from Bao!'

INFO  [05:03:05] (+1739212ms) core.syncTime[time.go:34] - clock offset 14.305609ms from time.google.com 
INFO  [05:33:05] (+3539225ms) core.syncTime[time.go:34] - clock offset 11.405685ms from time.google.com 
INFO  [06:03:05] (+5339250ms) core.syncTime[time.go:34] - clock offset 30.667482ms from time.google.com 
INFO  [06:33:05] (+7139320ms) core.syncTime[time.go:34] - clock offset 12.383112ms from time.google.com 
INFO  [07:28:01] (+8939224ms) core.syncTime[time.go:34] - clock offset 10.421456ms from time.google.com 
INFO  [09:18:50] (+10739221ms) core.syncTime[time.go:34] - clock offset 8.999673ms from time.google.com 
INFO  [09:50:28] (+12539222ms) core.syncTime[time.go:34] - clock offset 13.493494ms from time.google.com 
INFO  [10:20:28] (+14339407ms) core.syncTime[time.go:34] - clock offset 12.517736ms from time.google.com 
INFO  [10:50:28] (+16139282ms) core.syncTime[time.go:34] - clock offset 46.639947ms from time.google.com 
INFO  [11:20:28] (+17939219ms) core.syncTime[time.go

## Metadata
The write API supports metadata.

In [29]:
import json
vault2.write('docs/hello-attrs.txt', attrs=json.dumps({'edited_by': 'bob'}).encode('utf-8'))

file = vault2.stat('docs/hello-attrs.txt')
print('File attributes:', json.loads(file.attrs.decode('utf-8')))

File attributes: {'edited_by': 'bob'}


INFO  [04:25:45] (+0ms) vault.(*Vault).Stat[stat.go:40] - successfully got file info for docs/hello-attrs.txt: id=4, modTime=2026-01-31 04:25:45.684 +0100 CET, size=0, allocated=168, isDir=false


## Messaging
Send and receive `Message` objects inside a vault directory.


In [17]:
mailbox = Mailbox(vault, 'mailbox')
mailbox.send(Message('Greetings', 'Hello team!'))
mailbox.receive(0, 0)

[Message(subject='Greetings', body='Hello team!', attachments=[], fileInfo={'id': 3, 'name': '5a945043f1a7000', 'realm': 'users', 'size': 0, 'allocatedSize': 392, 'modTime': '2026-01-30T17:20:36.195+01:00', 'isDir': False, 'flags': 0, 'attrs': 'eyJzdWJqZWN0IjoiR3JlZXRpbmdzIiwiYm9keSI6IkhlbGxvIHRlYW0hIiwiYXR0YWNobWVudHMiOltdLCJmaWxlSW5mbyI6eyJpZCI6MCwibmFtZSI6IiIsInJlYWxtIjoiIiwic2l6ZSI6MCwiYWxsb2NhdGVkU2l6ZSI6MCwibW9kVGltZSI6IjAwMDEtMDEtMDFUMDA6MDA6MDBaIiwiaXNEaXIiOmZhbHNlLCJmbGFncyI6MCwia2V5SWQiOjAsInN0b3JlRGlyIjoiIiwic3RvcmVOYW1lIjoiIiwiYXV0aG9ySWQiOiIifX0=', 'keyId': 407932951282216960, 'storeDir': 'users/data/20260130000000', 'storeName': '6iywdzq8', 'authorId': 'A2bE9-oSjsPM04PaRqm2-kMepZo-pl3aWDbA5rjU_qq-mTDCBO4LUorBF_j7vyTxvQXl-uI9_4-SrwkPytl6txI='})]

## Replica interface
Attach a SQL interface that replicates statements through the vault. The interface allows placeholders for the SQL expressions.
The placeholders are defined in the statementes that creates the DB.
Each statements is introduced by a comment:
- a comment starting with _INIT_ and _version_ number is used during the creation of the DB
- other comments are placeholders for the queries

In [18]:
ddl = '''
-- INIT 1.0
CREATE TABLE IF NOT EXISTS notes (id INTEGER PRIMARY KEY, body TEXT);

-- INSERT_NOTE 1.0
INSERT INTO notes (id, body) VALUES (:id, :body);

-- SELECT_ALL_NOTES 1.0
SELECT * FROM notes;
'''

db_path = temp_root / 'data.db'
if db_path.exists():
    db_path.unlink()
    
data_db = DB('sqlite3', str(db_path), ddl=ddl)
replica = Replica(vault, data_db)
replica.exec('INSERT_NOTE', {'id': 1, 'body': 'hello'})
replica.exec('INSERT_NOTE', {'id': 2, 'body': 'bao'})
replica.fetch('SELECT_ALL_NOTES', {})

replica.sync()

INFO  [17:20:41] (+2ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data.db with driver sqlite3


2

Another peer can receive the updates

In [19]:
db_path2 = temp_root / 'data2.db'
if db_path2.exists():
    db_path2.unlink()
data_db2 = DB('sqlite3', str(db_path2), ddl=ddl)
replica2 = Replica(vault2, data_db2)
replica2.sync() 
replica2.fetch('SELECT_ALL_NOTES', {})

INFO  [17:20:46] (+1ms) sqlx.Open[db.go:63] - successfully opened SQLite db /var/folders/ns/9tw5wcmx4jd5ymdwq90bhk800000gp/T/bao_python_sample/data2.db with driver sqlite3


[[1, 'hello'], [2, 'bao']]

INFO  [17:42:12] (+1286067ms) core.syncTime[time.go:34] - clock offset 13.260081ms from time.google.com 
